This is an edited version of David NQ's Halite Game Scraper at https://www.kaggle.com/david1013/halite-game-scraper

Kaggle's API limit for Google Football is yet to be made explicit but in Kaggle Halite the limit of 1000 requests per day was eventually raised to 3600 requests per day max.

Rate limits are shared between the ListEpisodes and GetEpisodeReplay endpoints. Exceeding limits repeatedly will lead to temporary and then permanent bans. At some point it is expected Kaggle will remove this public API and provide datasets of episodes.

The episodes take a lot of space. In Kaggle Halite, I ended up with 200GB of games. The Football JSON files are **ten times larger** so you may end up with terabytes. If you use this or any scraper, consider posting the dataset to Kaggle Datasets for others to use.

In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime
import time

In [25]:
MIN_FINAL_RATING = 1300 # top submission in a match must have reached this score
num_api_calls_today = 0

In [26]:
if not os.path.isdir('./episodes'):
    os.mkdir('./episodes/')

In [27]:
all_files = []
for root, dirs, files in os.walk('./episodes/', topdown=False):
    all_files.extend(files)
seen_episodes = [int(f.split('.')[0]) for f in all_files 
                      if '.' in f and f.split('.')[0].isdigit() and f.split('.')[1] == 'json']
print('{} games in existing library'.format(len(seen_episodes)))

0 games in existing library


In [28]:
NUM_TEAMS = 1
EPISODES = 600 

BUFFER = 1

base_url = "https://www.kaggle.com/requests/EpisodeService/"
get_url = base_url + "GetEpisodeReplay"
list_url = base_url + "ListEpisodes"

In [29]:
# inital team list

r = requests.post(list_url, json = {"teamId":  5586412}) # arbitrary ID, change to leading ID during challenge

rj = r.json()

teams_df = pd.DataFrame(rj['result']['teams'])

In [30]:
teams_df.sort_values('publicLeaderboardRank', inplace = True)
teams_df.head()

,id,teamName,competitionId,teamLeaderId,isBenchmark,createdAfterDeadline,medal,dateMedalAwarded,submissionCount,lastSubmissionDate,publicLeaderboardSubmissionId,publicLeaderboardScore,publicLeaderboardRank,privateLeaderboardSubmissionId,privateLeaderboardScore,privateLeaderboardRank,competition,teamMembers,inboundMergeRequests,outboundMergeRequests
264,5673355,Win or Go Home,21723,1565175,False,False,0,None,45,"{'seconds': 1605535752, 'nanos': 760000000}",18061225,None,3,None,None,None,None,"[{'id': 9602, 'displayName': 'ForeverDream', '...",[],[]
235,5645993,s_toppo,21723,2304617,False,False,0,None,120,"{'seconds': 1605587850, 'nanos': 373333300}",18039596,None,5,None,None,None,None,"[{'id': 1958238, 'displayName': 'e-toppo', 'em...",[],[]
55,5588536,WFC,21723,1297333,False,False,0,None,76,"{'seconds': 1605356153, 'nanos': 406666700}",18025497,None,6,None,None,None,None,"[{'id': 1297333, 'displayName': 'NQ', 'email':...",[],[]
85,5590149,kangaroo,21723,5860849,False,False,0,None,17,"{'seconds': 1605593839, 'nanos': 216666700}",18079550,None,7,None,None,None,None,"[{'id': 831173, 'displayName': 'zlw21gxy', 'em...",[],[]
34,5587512,PC,21723,989207,False,False,0,None,55,"{'seconds': 1605574987, 'nanos': 596666700}",17936094,None,8,None,None,None,None,"[{'id': 832893, 'displayName': 'Xin Cui', 'ema...",[],[]


In [32]:
def getTeamEpisodes(team_id):
    # request
    r = requests.post(list_url, json = {"teamId":  int(team_id)})
    rj = r.json()

    # update teams list
    global teams_df
    teams_df_new = pd.DataFrame(rj['result']['teams'])
    
    if len(teams_df.columns) == len(teams_df_new.columns) and (teams_df.columns == teams_df_new.columns).all():
        teams_df = pd.concat( (teams_df, teams_df_new.loc[[c for c in teams_df_new.index if c not in teams_df.index]] ) )
        teams_df.sort_values('publicLeaderboardRank', inplace = True)
    else:
        print('teams dataframe did not match')
    
    # make df
    team_episodes = pd.DataFrame(rj['result']['episodes'])
    team_episodes['avg_score'] = -1;
    
    for i in range(len(team_episodes)):
        agents = team_episodes['agents'].loc[i]
        agent_scores = [a['updatedScore'] for a in agents if a['updatedScore'] is not None]
        team_episodes.loc[i, 'submissionId'] = [a['submissionId'] for a in agents if a['submission']['teamId'] == team_id][0]
        team_episodes.loc[i, 'updatedScore'] = [a['updatedScore'] for a in agents if a['submission']['teamId'] == team_id][0]
        
        if len(agent_scores) > 0:
            team_episodes.loc[i, 'avg_score'] = np.mean(agent_scores)

    for sub_id in team_episodes['submissionId'].unique():
        sub_rows = team_episodes[ team_episodes['submissionId'] == sub_id ]
        max_time = max( [r['seconds'] for r in sub_rows['endTime']] )
        final_score = max( [r['updatedScore'] for r_idx, (r_index, r) in enumerate(sub_rows.iterrows())
                                if r['endTime']['seconds'] == max_time] )

        team_episodes.loc[sub_rows.index, 'final_score'] = final_score
        
    team_episodes.sort_values('avg_score', ascending = False, inplace=True)
    return rj, team_episodes

In [33]:
def saveEpisode(epid, rj):
    # request
    re = requests.post(get_url, json = {"EpisodeId": int(epid)})
        
    # save replay
    with open('./episodes/{}.json'.format(epid), 'w') as f:
        f.write(re.json()['result']['replay'])

    # save episode info
    with open('./episodes/{}_info.json'.format(epid), 'w') as f:
        json.dump([r for r in rj['result']['episodes'] if r['id']==epid][0], f)

In [ ]:
global num_api_calls_today

pulled_teams = {}
pulled_episodes = []
failed_episodes = []
start_time = datetime.datetime.now()
r = BUFFER;

while num_api_calls_today < EPISODES:
    # pull team
    top_teams = [i for i in teams_df.id if i not in pulled_teams and i not in failed_episodes]
    if len(top_teams) > 0:
        team_id = top_teams[0]
    else:
        break;
        
    # get team data
    r += 1
    num_api_calls_today += 1
    try:
        team_json, team_df = getTeamEpisodes(team_id)
    except:
        failed_episodes.append(team_id)
        continue
        
    print('{} games for {}'.format(len(team_df), teams_df.loc[teams_df.id == team_id].iloc[0].teamName))

    
    team_df = team_df[  (MIN_FINAL_RATING is None or (team_df.final_score > MIN_FINAL_RATING))]
    
    print('   {} in score range from {} submissions'.format(len(team_df), len(team_df.submissionId.unique() ) ) )
    
    team_df = team_df[~team_df.id.isin(pulled_episodes + seen_episodes)]        
    print('      {} remain to be downloaded\n'.format(len(team_df)))
        
    # pull games
    target_team_games = int(np.ceil(EPISODES / NUM_TEAMS))
    if target_team_games + len(pulled_episodes) > EPISODES:
        target_team_games = EPISODES - len(pulled_episodes)
     
    pulled_teams[team_id] = 0
    
    i = 0
    while i < len(team_df) and pulled_teams[team_id] < target_team_games:
        epid = team_df.id.iloc[i]
        if not (epid in pulled_episodes or epid in seen_episodes):
            try:
                saveEpisode(epid, team_json); r+=1;
                num_api_calls_today+=1
            except:
                time.sleep(20)
                i+=1;
                continue;
            pulled_episodes.append(epid)
            pulled_teams[team_id] += 1
            try:
                size = os.path.getsize('./episodes/{}.json'.format(epid)) / 1e6
                print(str(num_api_calls_today) + ': Saved Episode #{} @ {:.1f}MB'.format(epid, size))
            except:
                print('  file {}.json did not seem to save'.format(epid))    
            if r > (datetime.datetime.now() - start_time).seconds:
                time.sleep( r - (datetime.datetime.now() - start_time).seconds)
        i+=1;
    print()
    print()

teams dataframe did not match
